In [1]:
from pyspark.sql import SparkSession

try:
    # Create a Spark session
    spark = SparkSession.builder.appName("ParquetReader").getOrCreate()

    # Continue with the rest of your script...
    # ...

except Exception as e:
    print("Error:", str(e))
    # Print additional diagnostic information
    print("Check Spark installation, resource availability, port availability, and firewall settings.")
    # Print Spark configuration for additional insights
    print("Spark Configuration:")
    print(spark.sparkContext.getConf().getAll())

    # Stop the Spark session if it was partially created
    spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/11 23:15:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

#Specify the paths to the Parquet files
covid_data_path = "data/covid_dataset.parquet"
hospital_data_path = "data/hospital_data/hospital_data.parquet"

# Read the parquet file into the dataframes
hospital_df = spark.read.parquet(hospital_data_path)

hospital_df.show(10)

+--------------------+------------+--------------+--------------------+-----------------------+
|       Hospital_Name|        City|Beds_Available|COVID_Beds_Available|NonCOVID_Beds_Available|
+--------------------+------------+--------------+--------------------+-----------------------+
|    Pune Hospital 74|        Pune|            73|                  18|                     55|
|Hyderabad Hospita...|   Hyderabad|            86|                  78|                      8|
|  Mumbai Hospital 30|      Mumbai|            69|                  30|                     39|
|  Mumbai Hospital 56|      Mumbai|           166|                  73|                     93|
|  Mumbai Hospital 66|      Mumbai|            56|                  48|                      8|
|   Delhi Hospital 84|       Delhi|            97|                  87|                     10|
|    Delhi Hospital 6|       Delhi|           162|                  93|                     69|
|Bhubaneshwar Hosp...|Bhubaneshwar|     

In [3]:

covid_data_path = "data/covid_dataset.parquet"
# Read the parquet file into the dataframes
covid_df = spark.read.parquet(covid_data_path)

covid_df.show(10)


+----------+----------------+--------------+-------+------------+
|Patient_ID|Health_Condition|Vaccine_Status|Country|        City|
+----------+----------------+--------------+-------+------------+
|         1|        Critical|      Received|  India|      Bhopal|
|         2|        Critical|      Received|  India|       Delhi|
|         3|        Critical|      Received|  India|     Chennai|
|         4|        Critical|      Received|  India|       Noida|
|         5|        Critical|      Received|  India|Bhubaneshwar|
|         6|        Critical|      Received|  India|      Bhopal|
|         7|        Critical|      Received|  India|      Jhansi|
|         8|        Critical|      Received|  India|      Mumbai|
|         9|        Critical|      Received|  India|      Bhopal|
|        10|        Critical|      Received|  India|     Chennai|
+----------+----------------+--------------+-------+------------+
only showing top 10 rows



In [5]:
from pyspark.sql import SparkSession

# Create a Spark session

joined_df = covid_df.join(hospital_df,'City')
joined_df.show(10)

+------------+----------+----------------+--------------+-------+--------------------+--------------+--------------------+-----------------------+
|        City|Patient_ID|Health_Condition|Vaccine_Status|Country|       Hospital_Name|Beds_Available|COVID_Beds_Available|NonCOVID_Beds_Available|
+------------+----------+----------------+--------------+-------+--------------------+--------------+--------------------+-----------------------+
|      Bhopal|         1|        Critical|      Received|  India|  Bhopal Hospital 53|            67|                  62|                      5|
|      Bhopal|         1|        Critical|      Received|  India|  Bhopal Hospital 22|           143|                  42|                    101|
|      Bhopal|         1|        Critical|      Received|  India|  Bhopal Hospital 62|            97|                  14|                     83|
|       Delhi|         2|        Critical|      Received|  India|    Delhi Hospital 6|           162|                 

In [ ]:
#Write this dataframe into the csv file
joined_df.write.csv(csv_output_path)
